In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import dataframe_image as dfi
from shapely import wkt
import sys
# !{sys.executable} -m pip install contextily
import contextily as ctx 
import matplotlib as mpl
from pandas import ExcelWriter


/usr/local/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
FilePath = '/Users/hoperuse/summer2021internship/State Surplus/final_data.geojson' # Replace with your own file path to csv of DOT parcels

# Data - Pandas DataFrame holding all parcel data in the final data universe, dropping leftover garbarge index columns
data = gpd.read_file(FilePath,index=False).drop(columns=['index'])

# Removing the word "District" from the column holding the name of the corresponding districts for each parcel
# to prep for geomerge with State House district lines shapefile.
data['District'] = data['District'].str.replace(' District','')

data = data.rename(columns={'District': 'REP_DIST'})

data['lot_area_acres'] = data['lot_areaft']/43560

# subsetData - filtered dataset of just those parcels with land use codes of 973 (vacant, DOH) and 975 (vacant, DOT)
subsetData = data[data['luc_adj_2'].isin(['973','975'])] 

# MA House Leg Districts (https://www.mass.gov/info-details/massgis-data-massachusetts-house-legislative-districts#downloads-)
houseDistricts = gpd.read_file('/Users/hoperuse/summer2021internship/State Surplus/house2012/HOUSE2012_POLY.shp')

In [3]:
pd.reset_option('^display.', silent=True)

In [4]:
data_districts = data.groupby('REP_DIST')

In [5]:
with pd.ExcelWriter('/Users/hoperuse/summer2021internship/State Surplus/district_breakdowns.xlsx') as writer:
    for district in list(data_districts.groups.keys()):
        df = data_districts.get_group(district)
        df.to_excel(writer, sheet_name=district,index=False)

In [6]:
from pandas import ExcelWriter
pd.options.display.float_format = '{:20,.2f}'.format

with pd.ExcelWriter('/Users/hoperuse/summer2021internship/State Surplus/district_totals.xlsx') as writer:
    for district in list(data_districts.groups.keys()):
        df = data_districts.get_group(district)[['land_value','lot_areaft','lot_area_acres','numTransitStops','luc_adj_2']]
        number_parcels = pd.Series(data = {0: len(df.index)}, name = 'number_parcels')
        number_transit_friendly_parcels = pd.Series(data = { 0: len(df[df.numTransitStops.notna()])},name = 'number_transit_parcels')
        number_ID_parcels = pd.Series(data = {0: len(df[df.luc_adj_2.isin(['973','975'])].index)}, name = 'number_ID_parcels')
        df = pd.DataFrame(df.sum())
        df = df.append([number_parcels,number_transit_friendly_parcels,number_ID_parcels])
        df = df.drop(index = ['numTransitStops', 'luc_adj_2'])
        df.to_excel(writer, sheet_name = district, index = True)


In [66]:
allParcels = data.pivot_table(index='REP_DIST', values=['mapc_id','land_value','lot_areaft','lot_area_acres','numTransitStops'], aggfunc={'mapc_id':'count','land_value':sum,'lot_areaft':sum, 'lot_area_acres':sum,'numTransitStops':'count'}).reset_index()

file = open("report.txt", "w")

file.write("ALL PARCELS: \n")
file.write("Total Number of Parcels: " + str(len(data.index)) + "\n")
file.write("Total Land Area of All Parcels (sq ft): "+ str(data.lot_areaft.sum()) + '\n')
file.write("Total Land Area of All Parcels (acres): " + str(data.lot_area_acres.sum()) + "\n")
file.write("Total Land Value of All Parcels: " + '$' + str(data.land_value.sum()) + "\n")
file.write("Total Number of All Parcels with Transit Stops: " + str(len(data[data.numTransitStops.notna()])) + "\n")

subsetParcels = subsetData.pivot_table(index='REP_DIST', values=['mapc_id','land_value','lot_areaft','lot_area_acres','numTransitStops'], aggfunc={'mapc_id':'count','land_value':sum,'lot_areaft':sum, 'lot_area_acres':sum,'numTransitStops':'count'}).reset_index()

file.write('\nSUBSET OF PARCELS: ' + str(subsetData.luc_adj_2.unique().flatten()) + "\n")
file.write("Total Number of Subset Parcels: " + str(len(subsetData.index)) + "\n")
file.write("Total Land Area of Subset Parcels (sq ft): " +  str(subsetData.lot_areaft.sum()) + "\n")
file.write("Total Land Area of Subset Parcels (acres): " + str(subsetData.lot_area_acres.sum()) + "\n")
file.write("Total Land Value of Subset Parcels: " + '$' + str(subsetData.land_value.sum()) + "\n")
file.write("Total Number of Subset Parcels with Transit Stops: "+ str(len(subsetData[subsetData.numTransitStops.notna()])) + '\n')
file.close()

In [67]:
pd.options.display.float_format = '{:.2f}'.format

with pd.ExcelWriter('/Users/hoperuse/summer2021internship/State Surplus/rankings.xlsx') as writer:
    for value in ['mapc_id','land_value','lot_area_acres']:
        df1 = allParcels.sort_values(by=value,ascending=False).reset_index(drop=True)
        df1 = df1[['REP_DIST', value]]

        df2 = subsetParcels.sort_values(by=value,ascending=False).reset_index(drop=True)
        df2 = df2[['REP_DIST', value]] 

        if value == 'lot_area_acres':
            df1[value] = df1[value].astype(int)
            df2[value] = df2[value].astype(int)

        df1.to_excel(writer, sheet_name=value + '_all_parcels',index=False)
        df2.to_excel(writer, sheet_name=value + '_subset_parcels',index=False)
       

In [68]:
members = ['David Henry Argosky Leboeuf (D)','James Arciero (D)','John H. Rogers (D)','Christopher Hendricks (D)','Patrick Joseph Kearney (D)','Mike Connolly (D)','Rob Consalvo (D)','Nika Elugardo (D)','Kip A. Diggs (D)','David F. DeCoste (R)','David K. Muradian, Jr. (R)']

In [71]:
import pathlib
for member in members:
    new_dir = pathlib.Path('/Users/hoperuse/summer2021internship/State Surplus/district_maps/', '_'.join(member.split(' ')[:-1]))
    new_dir.mkdir(parents=True, exist_ok=True)

    district = houseDistricts[houseDistricts.REP == member]['REP_DIST'].values[0]
    
    base_map = houseDistricts[houseDistricts.REP == member]

    ax = base_map.plot(figsize=(10, 10), alpha=0.5, edgecolor='k')

    parcel_map = data[data.REP_DIST == district]

    parcel_map.plot(ax=ax, marker='o', color='red', markersize=20,zorder=2)
    ctx.add_basemap(ax,zoom=12)
    ax.set_axis_off()

    plt.savefig('/Users/hoperuse/summer2021internship/State Surplus/district_maps/' + '_'.join(member.split(' ')[:-1]) + '/' + district.replace(' ', '_') + '_map.png')
    plt.close()

    all_parcels_district = allParcels[allParcels.REP_DIST == district].reset_index(drop=True)
    all_parcels_district = all_parcels_district.rename(columns = {'land_value': 'Total Land Value', 'lot_area_acres': 'Total Area (Acres)', 'mapc_id':'Total # Parcels','numTransitStops':'# Transit Friendly Parcels'}).transpose()
    all_parcels_district.to_csv('/Users/hoperuse/summer2021internship/State Surplus/district_maps/' + '_'.join(member.split(' ')[:-1]) + '/' + district.replace(' ', '_') + '_all_parcels.csv')

    subset_parcels_district = subsetParcels[subsetParcels.REP_DIST == district].reset_index(drop=True)
    subset_parcels_district = subset_parcels_district.rename(columns = {'land_value': 'Total Land Value', 'lot_area_acres': 'Total Area (Acres)', 'mapc_id':'Total # Parcels','numTransitStops':'# Transit Friendly Parcels'}).transpose()
    subset_parcels_district.to_csv('/Users/hoperuse/summer2021internship/State Surplus/district_maps/' + '_'.join(member.split(' ')[:-1]) + '/' + district.replace(' ', '_') + '_subset_parcels.csv')

/usr/local/lib/python3.8/site-packages/geopandas/plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
/usr/local/lib/python3.8/site-packages/geopandas/plotting.py:678: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  warnings.warn(
